# Polynomial Regression
The first question I needed to ask myself is why decide to use Polynomial Regression and what does it do exactly? So according to the visualizations from the <a href="https://github.com/lynstanford/machine-learning-projects/machine-learning/multiple_regression.ipynb">Linear Regression</a> notebook, the connection between the dependent and independent variables appear to be strongly linear in relationship although the connection between the 'Volume' of daily bitcoin bought and sold has less association with the daily 'Close' price. 

Fitting a Linear Regression line to the data may be accurate in this case, with an R-squared value of 0.9991392014437468 and RMSE of 689.1925598643533. However, out of curiosity I decided to see if a Polynomial function could fit the line slightly better by employing a regularization technique to try and improve the bias term by decreasing the Mean Squared Error.

The r-squared value is used to represent the overall accuracy score and directly measures the degree of variability associated between the predictors and target variable. The root mean squared value is represented as a loss function and my aim is to reduce its overall value as much as possible using regularization.

I know that Polynomial Regression is useful in determining non-linear relationships between multiple independent variables and the dependent variable, so it can be classified as a type of multiple linear regression. I can try to improve the fit of a prediction line to the data and improve estimates by changing the 'degree of fit' parameter, or by utilizing regularization.

## Import Data
Keeping the data loading simple this time will reduce the overall time it takes to retrieve.

In [ ]:
# import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# import data from filepath
btc_cad = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# storing dataset from filepath into new dataframe
bitcoin = pd.DataFrame(btc_cad).dropna(axis=0)

This reads the entire dataset and then stores all the values in a single dataframe object.

## Feature Selection and Scaling
Checking to see what features are present within the current dataframe.

In [ ]:
# all column names
bitcoin.columns

Now to remove the columns I don't need including 'Date' and 'Adj Close', selecting only the remaining ones to include in the X variable such as 'Open', 'High', 'Low' and 'Volume'. The remaining column of 'Close' in the same dataframe will be used as the target variable output, y.

In [ ]:
# remove feature with string values - Date
del bitcoin['Date']

# remove adj close as I will not be using this
del bitcoin['Adj Close']

# see the remaining features
bitcoin.columns

Assigning features to X gives:

In [ ]:
# select features as X dataframe
X = bitcoin[['Open','High','Low','Volume']]
print(X)

Comparing the shape of the overall dataset before polynomial regression and before splitting gives:

In [ ]:
X.shape

In [ ]:
# select target as y series
y = bitcoin['Close']
print(y)

In [ ]:
y.shape

Another way to select the right column vectors is using indexation.

In [ ]:
X = X.iloc[:,0:5].values
print(X[0:10])

In [ ]:
y = y.iloc[:, ].values
print(y[0:10])

Does the dataset need re-scaling? In this particular case the (X) predictors include 3 price variables and 1 volume variable which is scaled differently. The (y) target variable is another price variable, so the data would benefit from re-scaling. The transformation I have chosen will shift the values to a range between 0 and 1 for each column. 

Make a copy of the dataframe first.

In [ ]:
bitcoin = bitcoin.copy()

In [ ]:
# import preprocessing from sci-kit learn
from sklearn import preprocessing

# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
X_scaled = min_max_scaler.fit_transform(X)

bitcoin_features = pd.DataFrame(X_scaled)

bitcoin_features.to_csv(r'C:\Users\lynst\Documents\GitHub\machine-learning-projects\machine-learning\bitcoin_features.csv', index = False, header = True)

print(bitcoin_features)

Repeating this process for the y values:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

x = bitcoin_features.iloc[0:362,:1]
y = bitcoin_features.iloc[0:362,-1:]
z = bitcoin_features.iloc[0:362,3:4]

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(projection='3d')
ax.scatter(x, y, z, c='r', marker='o')
ax.set_xlabel("Open")
ax.set_ylabel("Close")
ax.set_zlabel("Volume")
plt.show()   

So for this particular comparison between the open and close prices and volume of Bitcoin transactions I can see a positive linear relationship between 'Open' and 'Close' prices. As one increases in value, so does the other. The relationship they have with 'Volume' appears somewhat linear also except for and outlier when volume spiked on Feb 26th, 2021. This was apparently due to bets by Tesla and Mastercard and stood out significantly compared to anything seen previously this year.

To display the outliers in the 'Volume' feature I've decided to use a boxplot.

In [ ]:
z = bitcoin_features.iloc[0:362,3:4]
z.head()

In [ ]:
fig = plt.figure(figsize=(10,8))
vol = z
vol.boxplot()

### Interquartile Range Method
To determine the outliers using an Interquartile range and remove them:

In [ ]:
# import the percentile package from numpy
from numpy import percentile

# convert the Series to a one-dimensional array
vol = np.array(z)

# calculate interquartile range
q25, q75 = percentile(vol, 25), percentile(vol, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off

# identify outliers
outliers = [v for v in vol if v < lower or v > upper]
print('Identified outliers: %d' % len(outliers))

# remove outliers
non_outliers = [v for v in vol if v >= lower and v <= upper]
print('Non-outlier observations: %d' % len(non_outliers))

### Standard Deviation Method
To determine outliers using the mean and standard deviation and removing any values which are outside 3 standard deviations from the mean:

In [ ]:
# identify outliers with standard deviation
from numpy import mean
from numpy import std

# convert the Series to a one-dimensional array
vol = np.array(z)

# calculate summary statistics
vol_mean, vol_std = mean(vol), std(vol)

# identify outliers
cut_off = vol_std * 3
lower, upper = vol_mean - cut_off, vol_mean + cut_off

# identify outliers
outliers = [v for v in vol if v < lower or v > upper]
print('Identified outliers: %d' % len(outliers))

# remove outliers
non_outliers = [v for v in vol if v >= lower and v <= upper]
print('Non-outlier observations: %d' % len(non_outliers))

### Automatic Outlier Detection Example


In [ ]:
from sklearn.model_selection import train_test_split

# retrieve the arrays
X = bitcoin[['Open','High','Low','Volume']].values
y = bitcoin['Close'].values

# split into input and output elements
X, y = bitcoin_features.iloc[0:362, 0:4], bitcoin_features.iloc[0:362, 4:5]

# summarize the shape of the dataset
print(X.shape, y.shape)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# evaluate model on training dataset with outliers removed
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error

# retrieve the arrays
X = bitcoin[['Open','High','Low','Volume']].values
y = bitcoin['Close'].values

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)

# identify outliers in the training dataset
lof = LocalOutlierFactor()
y_pred = lof.fit_predict(X_train)

# select all rows that are not outliers
mask = y_pred != -1
X_train, y_train = X_train[mask, :], y_train[mask]

# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)

# fit the model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# evaluate the model
y_pred = lin_reg.predict(X_test)

# evaluate predictions
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)

Having removed the outliers which fall outside the boundaries set by the LocalOutlierFactor( ) function, there appear to be 12 entries removed from the updated training set giving a total of 241 rows. It should be important to remember to use the LocalOutlierFactor( ) function and fit the data to the training set. 

The Root Mean Squared Error metric is:

In [ ]:
rmse = np.sqrt(mae)
rmse

Next I will perform both linear regression and polynomial regression to display prediction lines of best fit. I will only display the 'Close' price vs 'Date', so just two variables. I have chosen to manipulate data for price and time and store them in a new array called 'btc_new'.

In [ ]:
# import data from filepath
btc_cad = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")
# storing dataset from filepath into new dataframe
bitcoin = pd.DataFrame(btc_cad).dropna(axis=0)

I have decided to create a new data array containing the variables for date and the daily close price, having converted the dates to straight forward number of days (ranging from 1 to 362), the same as the total number of 'non-null' entries.

In [ ]:
# Visualising the Linear Regression results
figure = plt.figure(figsize=(14,8))

x = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,
              42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,
              80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,
              113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,
              141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,
              169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,
              197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,
              225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,
              253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,
              281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,
              309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,
              337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362])
y = np.array([9763.94,10096.28,10451.16,10642.81,10669.64,10836.68,10941.6,10917.12,12211.47,12083.24,12644.26,12820.88,12576.13,
              12551.25,12642.56,13128.23,13904.59,13795.97,13448.25,12194.49,12061.87,12397.94,13062.2,13659.96,13162.55,
              13228.95,13627.94,13559.83,13560.94,13241.13,12666.33,12857.9,12895.3,12293.22,12445.06,12177.47,12630.37,13121.03,
              13000.03,13359.9,13034.29,13812.49,12873.86,13031.1,13233.33,12972.79,12956.88,13071.93,13063.21,13147.35,13253.52,
              12711.01,12883.57,12876.0,12803.02,12791.0,12907.76,12871.53,12793.94,12640.25,12700.57,12668.61,13028.33,13041.35,
              12707.97,12634.47,12541.29,12380.43,12506.51,12552.25,12404.78,12543.14,12380.24,12313.75,12374.98,12296.15,
              12693.78,12588.0,12745.56,12599.85,12614.86,12556.24,12613.8,12580.1,12571.09,12417.11,12394.21,12428.28,12437.98,
              12469.29,12395.2,12604.21,12781.59,12843.04,12796.08,12982.28,13288.45,14662.38,14600.23,14809.4,14902.15,15186.41,
              15771.32,14809.83,15064.88,14900.92,15582.92,15690.25,15529.15,15733.03,15633.23,15862.41,15187.81,15342.8,
              15581.58,15614.35,15742.82,15761.58,16203.14,15775.92,15535.99,15648.99,15273.86,15391.56,15375.86,15563.07,
              14964.79,15097.17,14859.35,15119.23,15072.56,15319.24,15230.27,15626.5,14891.18,13462.52,13731.63,13284.56,
              13442.84,13579.41,13413.77,13471.4,13668.72,13705.81,13760.17,13609.86,14073.93,14244.94,14466.7,14398.2,14452.49,
              14650.47,14436.9,13916.7,14013.41,13690.02,14346.23,14325.02,14397.66,14417.81,14322.13,14520.83,14357.78,14115.09,
              14089.38,14063.2,14203.03,14325.21,14149.28,14160.03,14410.83,14818.74,14949.65,15031.95,15206.57,14936.71,
              14984.18,15137.27,15487.81,15634.23,16869.5,17032.64,16973.62,17205.31,17133.71,17267.45,18012.41,17666.01,
              17893.39,18045.32,18357.66,18364.88,17915.13,18279.6,18553.15,20383.97,20332.17,19375.87,20159.37,19945.73,
              19926.42,20504.46,21384.43,21423.16,21095.38,20937.96,21858.82,23126.07,23303.57,23320.17,24380.23,24407.62,
              24036.96,24010.26,24836.84,24356.4,22332.26,22226.46,23017.67,23600.83,25498.36,24319.8,24803.39,25023.04,23905.97,
              24486.96,24726.68,24572.39,23481.02,23800.7,23272.47,23059.65,24014.9,24409.37,24561.12,24658.5,27166.03,29036.47,
              29589.87,30525.81,30075.87,29308.0,30662.86,29851.39,30529.54,31754.68,34036.36,33737.04,34786.05,35085.89,
              36777.84,36919.19,37393.09,40898.17,41711.19,40865.06,43089.99,46641.22,49945.91,51769.03,51079.4,48817.74,45432.6,
              43108.25,47383.3,49563.35,46905.54,46081.14,45711.01,46701.33,45888.95,44892.29,38992.07,42028.71,40834.13,
              41094.11,41229.38,41341.23,39009.81,40608.72,43844.33,43794.73,42390.85,43093.59,45408.61,47908.07,47359.34,
              48683.77,50115.41,49642.27,58850.14,59023.47,57035.0,60845.81,60319.29,59816.94,61818.59,60583.38,62545.24,
              66229.13,65537.27,70533.62,70772.35,72505.56,68363.29,61493.78,62195.54,59404.52,59028.47,58830.69,57312.2,62739.6,
              61132.9,64055.45,61573.38,61913.1,61894.22,64684.29,66138.41,69333.05,70691.19,72463.92,71526.08,76406.88,73947.62,
              69760.45,70684.33,72931.8,72297.91,73079.59,73038.24,72043.42,68277.85,68917.98,66392.34,65160.43,69541.94,
              70596.59,70416.7,72713.56,74365.91,74029.63,74156.38,74675.77,72436.89,73827.42,73942.95,73156.52,70708.54,
              73273.84,72978.66,74918.53,75463.91,75243.42,79598.41,78995.98,79437.88,77018.32,75906.36,70374.91,69788.23,
              71477.76])

# The array of x values
reg_line = np.linspace(1, 362)

# Applying a linear fit
polymodel = np.poly1d(np.polyfit(x, y, deg=1))

plt.scatter(x, y, color = 'turquoise')
plt.plot(reg_line, polymodel(reg_line))
plt.show()

This would represent a highly positive linear relationship between price and time over the last year, specifically a high bias and low variance regression line, or an 'under-fitted' model. 

Making sure the values work printing out the first 10 values of each list.

In [ ]:
print(x[0:10])

In [ ]:
print(y[0:10])

Applying a polynomial degree of 2 this time provides an exponential curve. With a degree=2, the highest order value would be an exponent of 2, or x squared. The line used to capture these estimates are of low bias and low variance which is more suitable for prediction.

In [ ]:
figure = plt.figure(figsize=(14,8))

polymodel = np.poly1d(np.polyfit(x, y, deg=2))

reg_line = np.linspace(1, 362)

plt.scatter(x, y, color = 'turquoise')
plt.plot(reg_line, polymodel(reg_line))
plt.show()

Using a polyfit method with a degree of 50.

In [ ]:
figure = plt.figure(figsize=(14,8))

polymodel = np.poly1d(np.polyfit(x, y, deg=50, rcond=None, full=False, w=None, cov=False))

reg_line = np.linspace(1, 362, 362)

plt.scatter(x, y, color = 'turquoise')
plt.plot(reg_line, polymodel(reg_line))
plt.show()

# to eliminate the rank warning run this cell twice
import warnings
warnings.simplefilter('ignore', np.RankWarning)

In essence this represents 'over-fitting' and a low bias / high variance model.

## Select and Train a Model
Having introduced a pipeline to clean the data and apply machine learning algorithms automatically I've chosen to apply a Linear Regression model first. Once again, selecting the data to model for each variable, but this time using four independent variables as input predictors so the polynomial algorithm can be applied to try and fit the model slightly better than before.

In [ ]:
# import data from filepath
btc_cad = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# storing dataset from filepath into new dataframe
bitcoin = pd.DataFrame(btc_cad).dropna(axis=0)

# remove feature with string values - Date
del bitcoin['Date']

# remove adj close as I will not be using this
del bitcoin['Adj Close']

# select features as X dataframe
X = bitcoin[['Open','High','Low','Volume']]
print(X)

# select target as y series
y = bitcoin['Close']
print(y)

In [ ]:
print(X.shape)
print(y.shape)

## Scaling
Remember to apply the scaling because 'Volume' has different units of measurement.

In [ ]:
# making a copy of the dataset
bitcoin = bitcoin.copy()

# import preprocessing from sci-kit learn
from sklearn import preprocessing

# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
X_scaled = min_max_scaler.fit_transform(X)
coin_features = pd.DataFrame(X_scaled)
coin_features.to_csv(r'C:\Users\lynst\Documents\GitHub\machine-learning-projects\machine-learning\bitcoin_features.csv', index = False, header = True)

print(coin_features)

Having established a more comparible set of values for the entire dataset, I can proceed with application of my Polynomial model.

Before using the polynomial model I will train and fit a Linear Regression algorithm to learn from the entire dataset before the prediction or validation phase.

## Fit the Linear Regression Model

In [ ]:
# Fitting Linear Regression to the dataset
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

lin_reg = LinearRegression()
lin_reg.fit(X, y)

In [ ]:
# printing the r2 score
print(lin_reg.score(X, y))

Technically the dataset should be split into training and test sets so I can fit the model to the training set and learn from it, before validating the accuracy metric against the test set. If I fit the linear regression model to all data points, then split them into two different sets for training or testing, I would be introducing bias into the model.

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Re-running the r-squared score metric I get:

In [ ]:
print('R2: %0.8f' %r2_score(y_test, lin_reg.predict(X_test)))

This indicates that the model is generalizing well as the validation score is higher for the test set than the training set. This is a good sign! It could indicate a degree of data leakage because the score is so high, meaning it may not be the best representation of the correlation measure between the target and predictors because they're all extremely highly correlated.

To obtain a descriptive statistical summary I use the describe( ) function which provides figures I can use to calculate the R-squared manually....not just relying on the sklearn.metrics module.

In [ ]:
coin_features.describe()

I can check the R-squared value and plug in the inputs manually to see if it works.

Defining the intercept and coefficient values for x1, x2 ... xn from the linear regression before plugging in their values to the linear regression equation.

In [ ]:
lin_reg.intercept_

In [ ]:
lin_reg.coef_

Using the X values from the first row.

In [ ]:
X[:1]

Given this is linear regression and there are no polynomial terms I can calculate a prediction based on the following:

y = a + b1x1 + b2x2 + b3x3 + b4x4

Then, this will form the basis for my predictive model so I assign the values to a new variable called 'y_pred'.

In [ ]:
# linear regression terms for intercept and coefficients for row 1 values:
a = lin_reg.intercept_ = 104.15
b1 = lin_reg.coef_[0] = -0.43
b2 = lin_reg.coef_[1] = 0.92
b3 = lin_reg.coef_[2] = 0.51
b4 = lin_reg.coef_[3] = -0.0000000016
x1 = X['Open'] # = 9718.07
x2 = X['High'] # = 9838.33
x3 = X['Low'] # = 9728.25
x4 = X['Volume'] # = 46248330000.0
y = 9763.94
y_mean = 29864.75

Given the actual values, inetercept and coefficients (slopes), I can calculate predicted values.

In [ ]:
y_pred = a + (b1*x1) + (b2*x2) + (b3*x3) + (b4*x4)
y_pred = 104.15 + (-0.43 * 9718.07) + (0.92 * 9838.33) + (0.51 * 9728.25) + (-0.0000000016 * 46248330000.0)
print(y_pred)

The actual y value for the first row in the table was 9763.94. To calculate the R-squared score using the definitive statistical formula gives the following result.

In [ ]:
r2 = 1 - ((np.sum(y_pred - y)**2) / (np.sum(y_mean - y)**2))
print("The r-squared value is:", r2)

This is similar to the above score for the entire dataset but remember this is only for the first row in the table. Repeating this process for the entire dataset this time may require a little more thought as I need to iterate through all the values and integrate these inputs and define them as part of a new function:

In [ ]:
a = 104.15
b1 = -0.43
b2 = 0.92
b3 = 0.51
b4 = -0.0000000016
x1 = X['Open']
x2 = X['High']
x3 = X['Low']
x4 = X['Volume']

# obtaining the predicted values for y
def prediction():
    for i in range(len(X)):
        i = i + 1
        y_pred = a + (b1*x1) + (b2*x2) + (b3*x3) + (b4*x4)
    print(y_pred)
prediction()



In [ ]:
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# create actual and predicted values
y = bitcoin['Close']
actual = y

y_pred = a + (b1*x1) + (b2*x2) + (b3*x3) + (b4*x4)
predicted = y_pred

In [ ]:
actual[0:10]

In [ ]:
predicted[0:10]

In [ ]:
# create error values
error = actual - predicted
error = y - y_pred
errors = np.sum(error)
errors = list()
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)

def summary_stats():
    for i in range(len(actual)):
        error = actual - predicted
        error = y - y_pred
        errors = np.sum(error)
        errors = np.sum(y - y_pred)
        errors = list()
        mse = mean_squared_error(y, y_pred)
        rmse = np.sqrt(mse)
        
summary_stats()

print("Mean Squared Error is:", mse)
print("Root Mean Squared Error is:", rmse)
print("\n")

# plot errors
fig = pyplot.figure(figsize=(10,10))
pyplot.plot(errors)
pyplot.scatter(y, y_pred, color = 'coral')
pyplot.xlabel('Predicted Value')
pyplot.ylabel('Actual Value')
pyplot.show()

Having trained my model on predictor inputs, labeled targets and establishing a measure for the root mean squared error or the 'Loss Function', this can be used to compare the predictions to the targets. An important point to note is that it's better to better to perform scores on unseen test data that isn't used in the training phase. I can assess this measure as the level of risk or the degree of the dispersion of data observations about their mean. The r-squared score is the degree of accuracy of the linear relationship.

## Fit the Polynomial Model
Using the 'Open', 'High', 'Low' and 'Volume' input features and 'Close' as the output target can be modeled using the following Polynomial Regression formula for predicting y values:


$$  y = a + b1x1 + b2x2 + b3x3 + b4x4 + {(b5x1)^2} + {(b6x2)^2} + {(b7x3)^2} + {(b8x4)^2} + b9x1x2 + b10x1x3 + b11x1x4 + b12x2x3 + b13x2x4 + b14x3x4  $$


So I can see there are a total of 15 terms in this polynomial function which includes an intercept value (a) and several coefficients for each value of X, X-squared and the cross-multiplied combinations, cross-product, or vector-product of all features (x1, x2, x3, x4 etc).

In [ ]:
# Fitting Polynomial Regression followed by Linear Regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
poly.fit(X_poly, y)

There is no r-squared score for the polynomial model because the resultant training set (which is non-linear) has a different number of features than original.

## Split the Dataset
First I've decided to import the train_test_split function to split the dataset into respective training and test sets (for both X and y values in each set).

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=0)

## Applying Regularization to the Model
In this particular instance I have chosen L2 regularization which will penalize the Loss (or cost) function by summing the coefficients and equalizing features which exhibit strong colinearity, while simultaneously reducing the effect of redundant coefficients. (See the colinearity matrix in the previous <a href="https://github.com/lynstanford/machine-learning-projects/machine-learning/multiple_regression.ipynb">Linear Regression</a> model). For the purpose of this exercise the Root Mean Squared Error is used as the loss function.

Note, I have two hyperparameters I can adjust here including 'alpha' and 'normalize'. The 'alpha' value is the learning rate which controls the size of the incremental steps for the learning rate and the 'normalize' adjustment will re-scale the feature values between 0 and 1 for each observation in the dataset. An r-squared score will be used to determine its accuracy.

## Check the Accuracy Score for the Ridge Regression Fit
Using (L2) Ridge Regression first to see how close the relationship between X_train and y_train is, then validating against the test set.

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.1, normalize=True)
ridge_reg.fit(X_train, y_train)
print('R2: %0.8f' %r2_score(y_test, ridge_reg.predict(X_test)))

Adjusting the learning rate:

In [ ]:
ridge_reg = Ridge(alpha=0.01, normalize=True)
ridge_reg.fit(X_train, y_train)
print('R2: %0.8f' %r2_score(y_test, ridge_reg.predict(X_test)))

So changing the alpha learning rate from 0.1 to 0.01 implies I am increasing the number of steps it will take to reach the global minimum which means that convergence is slower, but also less likely to over-shoot the optimal solution. The accuracy score has dropped slightly as a result of lowering the alpha hyperparameter.

Having expanded the dataset using degree=2, a Polynomial Regression model does not have a score method so the dataset first has to be standardized using a linear model of some description. In this instance I am going to apply RidgeRegression( ) rather than straight forward LinearRegression( ) because the next important step is to standardize data using a regularization technique and reducing the scale of the units of measurement somewhere between -1 and +1, or between 0 and 1 depending on the scale I've chosen. Then the accuracy score can be applied after standardization.

## Re-scaling the Polynomial Dataset
Some of the values in this table would be far too large and exacerbate the effects of the model, so once again I have to scale the values and save the result as a csv file.

In [ ]:
# create a new copy of the dataset
X_poly = X_poly.copy()

# import preprocessing from sci-kit learn
from sklearn import preprocessing

# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
X_poly_scaled = min_max_scaler.fit_transform(X_poly)
X_poly_features = pd.DataFrame(X_poly_scaled)
X_poly_features.to_csv(r'C:\Users\lynst\Documents\GitHub\machine-learning-projects\machine-learning\X_poly_features.csv', index = False, header = True)

print(X_poly_features)

The shape of the new dataset with all the extended features is much larger and I can see a distinctive change in dimensions. Printing out the shape of the training set from the polynomial data gives:

In [ ]:
print(X_poly.shape)

Now for the training sets for X and y, 70% of the total values.

In [ ]:
print(X_train.shape)
print(y_train.shape)

And the shape of the test data.

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
ridge_reg = Ridge(alpha=0.1, solver="auto")
ridge_reg.fit(X_poly, y)
print('R2: %0.8f' %r2_score(y_test, ridge_reg.predict(X_test)))

In [ ]:
ridge_reg = Ridge(alpha=0.01, solver="auto")
ridge_reg.fit(X_poly, y)
print('R2: %0.8f' %r2_score(y_test, ridge_reg.predict(X_test)))

In [ ]:
X_poly_features.describe()

In [ ]:
X_poly_features[:1]

So looking into the first entry of a DataFrame object after applying regularization (L2, Ridge Regression) I can see that where a high degree of multi-colinearity exists between the features, the coefficients of those particular features are given a 'zero' value negating their effect on the model. It's the same as removing them completely. Any remaining coefficients which are positive in value (albeit negligable) stand out and obviously contribute more toward the calculation of the cost function, error calculation accuracy and optimization through iterative single-step gradient Descent.

In [ ]:
ridge_reg.intercept_

In [ ]:
ridge_reg.coef_

In [ ]:
X_poly[:1]

In [ ]:
# polynomial regression terms for intercept and coefficients for row 1 values:
a = ridge_reg.intercept_ = 11252.62
b1 = ridge_reg.coef_[0] = 0.00000000e+00
b2 = ridge_reg.coef_[1] = 2.20274036e-08
b3 = ridge_reg.coef_[2] = 1.01935648e-10
b4 = ridge_reg.coef_[3] = 9.51892565e-11
b5 = ridge_reg.coef_[4] = 3.60723223e-08
b6 = ridge_reg.coef_[5] = -7.32977764e-06
b7 = ridge_reg.coef_[6] = -7.82330374e-07
b8 = ridge_reg.coef_[7] = -9.48440941e-07
b9 = ridge_reg.coef_[8] = -2.11819215e-13
b10 = ridge_reg.coef_[9] = 5.91877653e-06
b11 = ridge_reg.coef_[10] = 5.75005955e-06
b12 = ridge_reg.coef_[11] = 2.53097632e-12
b13 = ridge_reg.coef_[12] = 5.46672010e-06
b14 = ridge_reg.coef_[13] = 1.44524432e-12
b15 = ridge_reg.coef_[14] = -4.88291757e-19
x1 = 1.0
x2 = X['Open']         # = 9718.07
x3 = X['High']         # = 9838.33
x4 = X['Low']          # = 9728.25
x5 = X['Volume']       # = 46248330000.0
x6 = (X['Open'])**2    # = (9.44408845e+07)**2
x7 = (X['High'])**2    # = (9.56095796e+07)**2
x8 = (X['Low'])**2     # = (9.45398145e+07)**2
x9 = (X['Volume'])**2  # = (4.49445461e+14)**2
x10 = x1 * x2          # = 9.67927372e+07
x11 = x1 * x3          # = 9.57097338e+07
x12 = x1 * x4          # = 4.55007297e+14
x13 = x2 * x3          # = 9.46388481e+07
x14 = x2 * x4          # = 4.49916270e+14
x15 = x3 * x4          # = 2.13891710e+21

y = 9763.94
y_mean = 29864.75

In [ ]:
y_pred = a + (b1*x1) + (b2*x2) + (b3*x3) + (b4*x4) + (b5*x5) + (b6*x6) + (b7*x7) + (b8*x8) + (b9*x9) + (b10*x10) + (b11*x11) + (b12*x12) + (b13*x13) + (b14*x14)
y_pred = 11252.62 + (0.0 * 1.0) + (2.20274036e-08 * 9.71807000e+03) + (1.01935648e-10 * 9.83833000e+03) + (9.51892565e-11 * 9.72825000e+03) + (3.60723223e-08 * 4.62484281e+10) + (-7.32977764e-06 * 9.44408845e+07) + (-7.82330374e-07 * 9.56095796e+07) + (-9.48440941e-07 * 9.45398145e+07) + (-2.11819215e-13 * 4.49445461e+14) + (5.91877653e-06 * 9.67927372e+07) + (5.75005955e-06 * 9.57097338e+07) + (2.53097632e-12 * 4.55007297e+14) + (5.46672010e-06 * 9.46388481e+07) + (1.44524432e-12 * 4.49916270e+14) + (-4.88291757e-19 * 2.13891710e+21)

print(y_pred)

Once again, comparing this predicted value against the actual y value for the first row in the table of 9763.94, there is quite a difference. To calculate the R-squared score using the definitive statistical formula gives the following result.

In [ ]:
r2 = 1 - ((np.sum(y_pred - y)**2) / (np.sum(y_mean - y)**2))
print("The r-squared value is:", r2)

In [ ]:
# create actual and predicted values
y = bitcoin['Close']
actual = y

y_pred = a + (b1*x1) + (b2*x2) + (b3*x3) + (b4*x4) + (b5*x5) + (b6*x6) + (b7*x7) + (b8*x8) + (b9*x9) + (b10*x10) + (b11*x11) + (b12*x12) + (b13*x13) + (b14*x14)
predicted = y_pred

In [ ]:
actual[0:10]

In [ ]:
predicted[0:10]

The negative effects of the coefficients and their small values have significantly reduced the squared-terms and the cross-product terms making them negligable.

In [ ]:
# import metric
from sklearn.metrics import mean_squared_error

# create error values
error = actual - predicted
error = y - y_pred
errors = np.sum(error)
errors = list()
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)

def summary_stats():
    for i in range(len(actual)):
        error = actual - predicted
        error = y - y_pred
        errors = np.sum(error)
        errors = list()
        mse = mean_squared_error(y, y_pred, squared=True)
        rmse = np.sqrt(mse)
    print(error)
    print('\n')
    print(mse)
    print(rmse)
    
summary_stats()

In [ ]:
print("Mean Squared Error is:", mse)
print("Root Mean Squared Error is:", rmse)
print("\n")

In [ ]:
# plot errors
fig = pyplot.figure(figsize=(10,10))
pyplot.plot(errors)
pyplot.scatter(y, y_pred, color = 'blue')
pyplot.xlabel('Predicted Value')
pyplot.ylabel('Actual Value')
pyplot.show()

The first note I can make about actual vs predicted daily closing prices leaves me to believe the errors have been reduced significantly about their mean, but I'm not entirely happy with the large value for the Cost function (RMSE) as this implies considerable over-fitting. One way to combat this challenge would be to introduce cross-validation which will take any outliers into account and remove some of the bias.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

ridge = linear_model.Ridge()
ridge.fit(X_train, y_train)
print(cross_val_score(ridge, X, y, cv=10))

In [ ]:
X_score = (0.97503778 + 0.87973022 + 0.98669072 + 0.96474476 + 0.98429722 + 0.99525415 + 0.99115778 + 0.89504007 + 0.94847447 + 0.90033073) / 10
print(X_score)

In [ ]:
print(cross_val_score(ridge, X_poly, y, cv=10))

In [ ]:
X_poly_score = (0.97006818 + 0.85076677 + 0.99061033 + 0.97345638 + 0.98865061 + 0.99535352 + 0.99329041 + 0.79100498 + 0.79524206 + 0.87280349) / 10
print(X_poly_score)

In [ ]:
print(cross_val_score(ridge, X_train, y_train, cv=10))

In [ ]:
X_train_score = (0.99859545 + 0.9982508 + 0.99873121 + 0.99971355 + 0.97387001 + 0.99905469 + 0.99721033 + 0.99879823 + 0.99922155 + 0.99974627) / 10
print(X_train_score)

The first important detail to note is that the average cross-validation score is higher for the training set than the entire dataset from ridge regression, or the polynomial dataset.

In [ ]:
from sklearn.metrics import explained_variance_score

mse = mean_squared_error(y_test, ridge.predict(X_test), squared=True)
mse_scores = cross_val_score(ridge, X_train, y_train, scoring="explained_variance", cv=10)
mse_scores

In [ ]:
average_mse = (0.99859549 + 0.99826071 + 0.99876091 + 0.9997251 + 0.97460254 + 0.99909548 + 0.99725663 + 0.99898202 + 0.99922302 + 0.99974633) / 10
print(average_mse)

And for the root mean squared error values, finding the average rmse:

In [ ]:
rmse_scores = np.sqrt(mse_scores)
rmse_scores

In [ ]:
average_rmse = (0.9992975 + 0.99912998 + 0.99938027 + 0.99986254 + 0.9872196 + 0.99954764 + 0.99862737 + 0.99949088 + 0.99961143 + 0.99987316) / 10
print(average_rmse)

Now to evaluate the r-squared scores by cross-validating ten different folds.

In [ ]:
from sklearn.metrics import r2_score

r2_scores_train = cross_val_score(ridge, X_train, y_train, scoring="r2", cv=10)
r2_scores_train

Using the results from r-squared scores first I can determine the average r2 value from 10 different folds.

In [ ]:
average_r2_train = (0.99859545 + 0.9982508 + 0.99873121 + 0.99971355 + 0.97387 + 0.99905469 + 0.99721033 + 0.99879823 + 0.99922155 + 0.99974627) / 10
print(average_r2_train)

To see how this model generalizes to the test data:

In [ ]:
r2_scores_test = cross_val_score(ridge, X_test, y_test, scoring="r2", cv=10)
r2_scores_test

In [ ]:
average_r2_test = (0.99984803 + 0.99985311 + 0.99813416 + 0.99927484 + 0.99985759 + 0.99957481 + 0.99982543 + 0.99799085 + 0.96522031 + 0.99736867) / 10
print(average_r2_test)

The scores seem to drop slightly for the test data but are approximately the same which is a good sign for the model because it means the fit, training and prediction phases will translate well to unseen data (generally) with at least 99.57% accuracy.

Another thing to notice is how the 'explained_variance_score' score between the X training values and y training values are similar to the 'r-squared' scores.

This particular 'cross_val_predict' metric is actually derived from the 'model_selection' model validation API and then, using indexing to return the first 5 values I get the following predictions:

In [ ]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(ridge, X, y, cv=10)
print(y_pred[0:5])

Cross validation is meant to determine the performance of different machine learning algorithms so they can be compared and contrasted. It's not meant for training a model, but for checking different models. I will apply the cross-validation on the dataset with all 4 different features (Open, High, Low and Volume) and a target column (Close). Starting from the beginning I will attempt to find the optimal value for K, or that which provides the highest accuracy, or the lowest MSE score.

The code below is used to formulate Stochastic Gradient Descent using the Learning Rate and Number of Iterations as arguments.

In [ ]:
alpha = 0.1
n_iterations = 1000
n = 100
n_epochs = 50
t0, t1 = 5, 50          # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)          # random initilaization

for epoch in range(n_epochs):
    for i in range(n):
        random_index = np.random.randint(n)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        alpha = learning_schedule(epoch * m + i)
        theta = theta - alpha * gradients
    
'''
# learning rate
alpha = 0.1
# parameter with default number of iterations in gradient descent
n_iterations = 1000
m = eta * n_iterations
X_T = np.transpose(X)
X_theta = X.dot(theta)

gradients = (2 / m) * X_T.dot(X_theta - y)
theta = theta - alpha * gradients
'''

In [ ]:
def gradient_descent(gradient, start, learn_rate, n_iter):
    vector = start
    for _ in range(n_iter):
        diff = -learn_rate * gradient(vector)
        vector += diff
    return vector

In [ ]:
import numpy as np

def gradient_descent(
    gradient, start, learn_rate, n_iter=50, tolerance=1e-06
):
    vector = start
    for _ in range(n_iter):
        diff = -learn_rate * gradient(vector)
        if np.all(np.abs(diff) <= tolerance):
            break
        vector += diff
    return vector

In [ ]:
gradient_descent(gradient=lambda v: 2 * v, start=10.0, learn_rate=0.2)

In [ ]:
gradient_descent(gradient=lambda v: 2 * v, start=10.0, learn_rate=0.8)

In [ ]:
gradient_descent(gradient=lambda v: 2 * v, start=10.0, learn_rate=0.005)

In [ ]:
def ssr_gradient(x, y, b):
    res = b[0] + b[1] * x - y
    return res.mean(), (res * x).mean()  # .mean() is a method of np.ndarray

In [ ]:
import numpy as np

def gradient_descent(
    gradient, x, y, start, learn_rate=0.1, n_iter=50, tolerance=1e-06
):
    vector = start
    for _ in range(n_iter):
        diff = -learn_rate * np.array(gradient(x, y, vector))
        if np.all(np.abs(diff) <= tolerance):
            break
        vector += diff
    return vector

In [ ]:
x = np.array([5, 15, 25, 35, 45, 55])
y = np.array([5, 20, 14, 32, 22, 38])

gradient_descent(ssr_gradient, x, y, start=[0.5, 0.5], learn_rate=0.0008, n_iter=100_000)

In [ ]:
import numpy as np

def gradient_descent(
    gradient, x, y, start, learn_rate=0.1, n_iter=50, tolerance=1e-06,
    dtype="float64"
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    if x.shape[0] != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")

    # Initializing the values of the variables
    vector = np.array(start, dtype=dtype_)

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Recalculating the difference
        diff = -learn_rate * np.array(gradient(x, y, vector), dtype_)

        # Checking if the absolute difference is small enough
        if np.all(np.abs(diff) <= tolerance):
            break

        # Updating the values of the variables
        vector += diff

    return vector if vector.shape else vector.item()

In addition to considering data types, the code above introduces a few modifications related to type checking and ensuring the use of NumPy capabilities:

Lines 8 and 9 check if gradient is a Python callable object and whether it can be used as a function. If not, then the function will raise a TypeError.

Line 12 sets an instance of numpy.dtype, which will be used as the data type for all arrays throughout the function.

Line 15 takes the arguments x and y and produces NumPy arrays with the desired data type. The arguments x and y can be lists, tuples, arrays, or other sequences.

Lines 16 and 17 compare the sizes of x and y. This is useful because you want to be sure that both arrays have the same number of observations. If they don’t, then the function will raise a ValueError.

Line 20 converts the argument start to a NumPy array. This is an interesting trick: if start is a Python scalar, then it’ll be transformed into a corresponding NumPy object (an array with one item and zero dimensions). If you pass a sequence, then it’ll become a regular NumPy array with the same number of elements.

Line 23 does the same thing with the learning rate. This can be very useful because it enables you to specify different learning rates for each decision variable by passing a list, tuple, or NumPy array to gradient_descent().

Lines 24 and 25 check if the learning rate value (or values for all variables) is greater than zero.

Lines 28 to 35 similarly set n_iter and tolerance and check that they are greater than zero.

Lines 38 to 47 are almost the same as before. The only difference is the type of the gradient array on line 40.

Line 49 conveniently returns the resulting array if you have several decision variables or a Python scalar if you have a single variable.

Your gradient_descent() is now finished. Feel free to add some additional capabilities or polishing. The next step of this tutorial is to use what you’ve learned so far to implement the stochastic version of gradient descent.

In [ ]:
import numpy as np

def sgd(
    gradient, x, y, start, learn_rate=0.1, batch_size=1, n_iter=50,
    tolerance=1e-06, dtype="float64", random_state=None
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]

    # Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    # Initializing the values of the variables
    vector = np.array(start, dtype=dtype_)

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Shuffle x and y
        rng.shuffle(xy)

        # Performing minibatch moves
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]

            # Recalculating the difference
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = -learn_rate * grad

            # Checking if the absolute difference is small enough
            if np.all(np.abs(diff) <= tolerance):
                break

            # Updating the values of the variables
            vector += diff

    return vector if vector.shape else vector.item()

In [ ]:
sgd(
...     ssr_gradient, x, y, start=[0.5, 0.5], learn_rate=0.0008,
...     batch_size=3, n_iter=100_000, random_state=0
... )

In [ ]:
import numpy as np

def sgd(
    gradient, x, y, start, learn_rate=0.1, decay_rate=0.0, batch_size=1,
    n_iter=50, tolerance=1e-06, dtype="float64", random_state=None
):
    # Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    # Setting up the data type for NumPy arrays
    dtype_ = np.dtype(dtype)

    # Converting x and y to NumPy arrays
    x, y = np.array(x, dtype=dtype_), np.array(y, dtype=dtype_)
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    xy = np.c_[x.reshape(n_obs, -1), y.reshape(n_obs, 1)]

    # Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    # Initializing the values of the variables
    vector = np.array(start, dtype=dtype_)

    # Setting up and checking the learning rate
    learn_rate = np.array(learn_rate, dtype=dtype_)
    if np.any(learn_rate <= 0):
        raise ValueError("'learn_rate' must be greater than zero")

    # Setting up and checking the decay rate
    decay_rate = np.array(decay_rate, dtype=dtype_)
    if np.any(decay_rate < 0) or np.any(decay_rate > 1):
        raise ValueError("'decay_rate' must be between zero and one")

    # Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    # Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    # Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")

    # Setting the difference to zero for the first iteration
    diff = 0

    # Performing the gradient descent loop
    for _ in range(n_iter):
        # Shuffle x and y
        rng.shuffle(xy)

        # Performing minibatch moves
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            x_batch, y_batch = xy[start:stop, :-1], xy[start:stop, -1:]

            # Recalculating the difference
            grad = np.array(gradient(x_batch, y_batch, vector), dtype_)
            diff = decay_rate * diff - learn_rate * grad

            # Checking if the absolute difference is small enough
            if np.all(np.abs(diff) <= tolerance):
                break

            # Updating the values of the variables
            vector += diff

    return vector if vector.shape else vector.item()

Because the loss function (RMSE) and the accuracy score (R-squared) can be improved upon, the next phase involves using an 'Optimization Function' to re-balance the coefficients of each data point reiterating through the data in an attempt to reduce the loss function. This will involve the adoption of optimization techniques such as 'Gradient Descent', particularly the SGDRegressor( ) algorithm.

## Stochastic Gradient Descent
Gradient descent provides an alternative way to measure 'Loss' which can be described as an iterative process of adjusting the parameters of a model, fine-tuning the gradient coefficients or using approximation to minimize the overall loss. SGD Regression is meant to be the same as Ridge Regression but perform slightly faster.

In [ ]:
# import the necessary models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score

# import data from filepath
btc_cad = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# storing dataset from filepath into new dataframe
bitcoin = pd.DataFrame(btc_cad).dropna(axis=0)

# select features as X dataframe
X = bitcoin[['Open','High','Low','Volume']]

# select target as y series
y = bitcoin['Close']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Store model in a variable
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.01)
sgd_reg.fit(X_train, y_train)

In [ ]:
r2 = 1 - ((np.sum(y_pred - y)**2) / (np.sum(y_mean - y)**2))
print("The r-squared value is:", r2)

In [ ]:
print(sgd_reg.score(X_test, y_test))

In [ ]:
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X_train, y_train)
print(sgd_reg.score(X_test, y_test))

Increasing the learning rate in the SGDRegressor has increased the score minimally but they're incredibly low. It would be best to introduce cross-validation to find the best achievable score for a pre-determined number of folds in the dataset.

First let's see the effect on Theta, the intercept and coefficient values:

In [ ]:
sgd_reg.intercept_

In [ ]:
sgd_reg.coef_

### Making a Prediction
Comparing predictions for training set and test set values. I am continuing to use the RMSE as the score measure to see if it can be optimized as far as possible for the purpose of this model.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, y_test_pred)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=y_test_pred)

# evaluating the model on train dataset
y_train = lin_reg.fit(y_train)
y_train_pred = lin_reg.predict(y_train)
y_test_pred = lin_reg.predict(y_test)

# evaluating the model on test dataset
mse_train = np.sum(y_train - y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_test_pred)
  
print("\n")
  
print("The model performance for the test set")
print("-------------------------------------------")

print("MSE of the train set is {}".format(mse_train))
print("RMSE of train set is {}".format(rmse_train))
print("R2 score of train set is {}".format(r2_train))

print("MSE of the test set is {}".format(mse_test))
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

print("\n")

I can deduce that the accuracy for determining predictions appears to be highest for plain linear regression, before any polynomial modeling is applied. This is probably due to the strong nature of the association between the price variables, or their degree of co-linearity.

## Predicting Against Test Set
Now to predict the test set results.

In [ ]:
y_test_pred = lin_reg.predict(X_test)

# to see the first ten results
y_test_pred[0:10]

Comparing the first row of X values to predict the y value by copying from the dataset output from cell 1. Remember only to use the 'Open', 'High', 'Low' and 'Volume' values.

In [ ]:
lin_reg.predict([[9718.07, 9838.33, 9728.25, 4.624843e+10]])

So the predicted y value is 9792.45 and the actual y value (from output of the first value for 'Close' price) is 9763.94, which is reasonably accurate.

## Model Evaluation
This is where I need to look into the root mean squared error and R-squared values to see how well the model is performing.

In [ ]:
# import the score measure
from sklearn.metrics import r2_score
r2_score(y_test, y_test_pred)

### Actual vs Predicted Prices
Plotting the actual and predicted results for the 'Close' price provides a nice visual display of the relationship.

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_test, y_test_pred, color="orange")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs. Predicted Prices")
plt.show()

## Predictions
Storing the actual and predicted values and their difference in a dictionary object becomes useful. The larger the error, the greater the variance and subsequent standard deviation which translates to less accurate predictions and a less reliable model. 

Why does this become important? The main purpose of this exercise is to see how useful my model would be in predicting the daily closing price so the first information I can derive from the scatter plot above shows the relationship between independent and dependent variables is strong, but also incredibly linear. It may not pay off to apply Polynomial Regression in such circumstances but may improve the linear fit to the data points with increasing degrees of freedom.

In [ ]:
predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_test_pred, "Error": y_test - y_test_pred})
predictions[0:10]

Exploring the entire dataset (all the X and y values) and the degree of accuracy by fitting a linear regression predictive line once more, but this time calling Seaborn's regression plot function (regplot).

In [ ]:
import seaborn as sns

# set the width and height of the plot
plt.figure(figsize=(15,10))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, y_test_pred, color="orange")
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=y_test_pred, color="orange")

Interpreting the RMSE tells me the results seem too high.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
# actual values
y_train = lin_reg.fit(y_train)

# predicted values
y_train_pred = lin_reg.predict(y_train)

# calculate errors
err = (y_train[i] - y_train_pred[i])**2
errors = list()

for i in range(len(y_train)):
    # calculate error
    err = (y_train[i] - y_train_pred[i])**2
    # store error
    errors.append(err)
    # report error
    print('>%.1f, %.1f = %.3f' % (y_train[i], y_train_pred[i], err))
# plot errors
plt.plot(errors)
plt.xticks(ticks=[i for i in range(len(errors))], labels=predicted)
plt.xlabel('Predicted Value')
plt.ylabel('Mean Squared Error')
plt.show()

In [ ]:
y_test_pred = ridge_regression.predict(X_test)
print("Predictions: ", ridge_regression.predict(X_test.iloc[:5]))

The regularization term should only be added to the cost function during the training phase. Now the 'training' data has been fit, it becomes important to discover the performance measure on the unregularized test set. Making a prediction on the first 5 values in the test set first.

In [ ]:
#lin_reg.predict([[363]])

In [ ]:
#lin_reg2.predict(poly.fit_transform([[363]]))

## Pipelines
To better capture the different features I have used in my multiple regression I will employ the use of a Pipeline to include the different types of regression models, re-scaling of the features with different units of measurement, regularization techniques to improve the degree of overall fit before validation.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler


pipeline = Pipeline([
    ('poly_reg', PolynomialFeatures(degree=2, include_bias=False)),
    ('lin_reg', LinearRegression(normalize=True)),
    ('scale', StandardScaler()),
    ('scores', cross_val_score(pipeline, X, y, scoring="neg_mean_squared_error", cv=5))
    ])

'''
scores = cross_val_score

pipeline.fit(X, y)
y_pred = pipeline.predict(X)
print(y_pred)
'''